# Ejercicio 50
Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o
igual a 1 consiga un dataframe cuyas columnas son tipos de logs, el índice es la
cantidad de idiomas que sabe un usuario y las celdas la probabilidad de que esos
usuarios generen ese tipo de log.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
dfLanguages = pd.read_csv('./WikipediaDataset/languages.csv')
dfLogs = pd.read_csv('./WikipediaDataset/logs.csv', nrows = 1000)

In [2]:
dfLogs

,item_id,timestamp,contributor_username,contributor_id,contributor_ip,comment,logtype,action,title
0,1,2004-12-23T07:10:09Z,Sabbut,2075,NaN,el contenido era: 'Si aprendés Pensamiento Cie...,delete,delete,Verificacionismo
1,2,2004-12-23T07:14:09Z,Sabbut,2075,NaN,el contenido era: '{{destruir}}en algun lugar ...,delete,delete,Rapunzel
2,3,2004-12-23T07:26:10Z,Sabbut,2075,NaN,el contenido era: '#REDIRECT [[Flash]]',delete,delete,Macromedia Flash
3,4,2004-12-23T07:55:09Z,Sanbec,7856,NaN,Sin información de origen ni de licencia. Hay ...,delete,delete,Archivo:Gnu head.png
4,5,2004-12-23T08:02:22Z,Sanbec,7856,NaN,Sustituida por [[commons:Image:Heckert_GNU.png]],delete,delete,Archivo:Gnu-head-sm.jpg
...,...,...,...,...,...,...,...,...,...
995,996,2005-01-02T23:01:06Z,Sabbut,2075,NaN,el contenido era: '{{copyvio|http://www.quiero...,delete,delete,Vilanova i la Geltrú (Barcelona)
996,997,2005-01-02T23:01:08Z,Sabbut,2075,NaN,NaN,delete,delete,Vilobí del Penedès (Barcelona)
997,998,2005-01-02T23:01:10Z,Sabbut,2075,NaN,el contenido era: 'Nombre de la página no corr...,delete,delete,Vínculo
998,999,2005-01-02T23:02:18Z,Sabbut,2075,NaN,el contenido era: '{{copyvio|http://www.shalom...,delete,delete,Yamim Noraim


In [3]:
dfLogs["logtype"].value_counts()

delete     763
upload     218
block       16
protect      3
Name: logtype, dtype: int64

In [4]:
dfLanguages

,babel_user,babel_lang,babel_level
0,4502458,FR,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0
...,...,...,...
35661,2089885,zh-Hant-HK,1
35662,2089885,zh-Hant-TW,3
35663,5036533,zu,0
35664,5928200,zu,0


In [5]:
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


In [6]:
dfLanguages = dfLanguages[dfLanguages['babel_lang'].notna()]
dfLanguages["babel_level"] = dfLanguages["babel_level"].replace("N", 4.5)

In [7]:
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35662 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35662 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35662 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [8]:
dfLanguages["babel_level"] = dfLanguages["babel_level"].astype(str).astype(float)
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35662 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   babel_user   35662 non-null  int64  
 1   babel_lang   35662 non-null  object 
 2   babel_level  35662 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


In [9]:
dfLanguages["sabe_el_lenguaje"] = dfLanguages["babel_level"] >= 1
dfLanguages 

,babel_user,babel_lang,babel_level,sabe_el_lenguaje
0,4502458,FR,2.0,True
1,5928200,aa,0.0,False
2,46918,ab,0.0,False
3,2050449,ab,1.0,True
4,4715583,ace,0.0,False
...,...,...,...,...
35661,2089885,zh-Hant-HK,1.0,True
35662,2089885,zh-Hant-TW,3.0,True
35663,5036533,zu,0.0,False
35664,5928200,zu,0.0,False


In [10]:
dfLanguages["cant_idiomas"] = dfLanguages.groupby("babel_user")["sabe_el_lenguaje"].transform("sum")
dfLanguages

,babel_user,babel_lang,babel_level,sabe_el_lenguaje,cant_idiomas
0,4502458,FR,2.0,True,3
1,5928200,aa,0.0,False,44
2,46918,ab,0.0,False,2
3,2050449,ab,1.0,True,7
4,4715583,ace,0.0,False,27
...,...,...,...,...,...
35661,2089885,zh-Hant-HK,1.0,True,10
35662,2089885,zh-Hant-TW,3.0,True,10
35663,5036533,zu,0.0,False,4
35664,5928200,zu,0.0,False,44


In [11]:
dfNoSabenLenguaje = dfLanguages.loc[~dfLanguages["sabe_el_lenguaje"], :]
dfNoSabenLenguaje

,babel_user,babel_lang,babel_level,sabe_el_lenguaje,cant_idiomas
1,5928200,aa,0.0,False,44
2,46918,ab,0.0,False,2
4,4715583,ace,0.0,False,27
5,5928200,ace,0.0,False,44
7,5928200,af,0.0,False,44
...,...,...,...,...,...
35462,5870407,zh,0.0,False,2
35463,5873480,zh,0.0,False,8
35464,6033821,zh,0.0,False,3
35663,5036533,zu,0.0,False,4


In [18]:
dfLanguages.loc[dfLanguages["cant_idiomas"] == 0]

,babel_user,babel_lang,babel_level,sabe_el_lenguaje,cant_idiomas
3905,4791887,en,0.0,False,0
13717,804126,es,0.0,False,0
13759,1206707,es,0.0,False,0
13788,1571823,es,0.0,False,0
13795,1714461,es,0.0,False,0
13864,4791887,es,0.0,False,0
13879,6089716,es,0.0,False,0
